In [1]:
from utils import time_op

In [4]:
# tests the model counting capabilities of SharpSAT-TD and Buddy
from formulas import GPMC, Formula, Buddy, random_k_cnf

n,m,k = 10, 10, 7
cnf, formula = random_k_cnf(n,m,k)
vars = [f"x{idx}" for idx in range(1, n+1)]

solver = GPMC()
satcount_gpmc = time_op(solver.satcount)(cnf)
print(f"GPMC satcount = {satcount_gpmc}")

with Buddy(vars) as model:
    f = time_op(model.node)(formula)
    satcount_bdd = time_op(lambda : f.satcount)()
    print(f"BDD satcount = {satcount_bdd}")


[00007.3519 ms / 0.0074 s / 0.0001 min] satcount
GPMC satcount = 961
[00079.2882 ms / 0.0793 s / 0.0013 min] node
[00000.0055 ms / 0.0000 s / 0.0000 min] <lambda>
BDD satcount = 961.0


In [5]:
n,clauses_to_variables,k = 20, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]

In [6]:
import importance_measures.bdds as bdd 
from formulas import Buddy

with Buddy(vars) as model:
    x = "x1"
    f = (time_op)(model.node)(formula)
    print(f"bdd size = {f.nodecount} with satcount = {f.satcount}")
    blame, ub = time_op(bdd.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
    infl = time_op(bdd.influence)(f, x)
    print(f"influence {infl:.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")

[01766.7472 ms / 1.7667 s / 0.0294 min] node
bdd size = 6707 with satcount = 541736.0
=== COMPUTING BLAME for x1 in BDD with size 6707 ===
k=0 size ell=3811 d result=0.1547 max increase possible=0.4227 
k=1 size ell=24991 d result=0.1671 max increase possible=0.1704 
k=2 size ell=24338 d result=0.1229 max increase possible=0.0356 
k=3 size ell=6707 d result=0.0315 max increase possible=0.0033 
k=4 size ell=810 d result=0.0032 max increase possible=0.0001 
stopped earlier because cannot improve above cutoff.
current value: 0.4793, can be increased by 0.0001 <= 0.0001.
=== DONE ===
[00425.4107 ms / 0.4254 s / 0.0071 min] blame
[00000.4506 ms / 0.0005 s / 0.0000 min] influence
influence 0.15466, 0.47933 <= blame <= 0.47942, blame diff 0.00009


In [7]:
import importance_measures.mc as mc
from formulas import GPMC, Formula

f = time_op(Formula.parse)(formula)
x = "x1"
solver = GPMC(cs=10000)
infl = (time_op)(mc.influence)(f, x, solver)
blame, ub = time_op(mc.blame)(f, x, solver, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
print(f"influence {infl:.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")

[02006.1674 ms / 2.0062 s / 0.0334 min] parse
[00080.7364 ms / 0.0807 s / 0.0013 min] influence
=== COMPUTING BLAME for x1 in Formula with size 2711 ===
k=0 size ell=19243 d result=87463.5000 max increase possible=-43731.2500 
stopped earlier because cannot improve above cutoff.
current value: 87463.5000, can be increased by -43731.2500 <= 0.0001.
=== DONE ===
[05762.0959 ms / 5.7621 s / 0.0960 min] blame
influence 0.15466, 87463.50000 <= blame <= 43732.25000, blame diff -43731.25000


In [6]:
import importance_measures.mc as mc 
from formulas import Formula, GPMC

f,_ = mc.at_most_cnf3(2, {1,2,3})
f = f + [[-1], [-2], [3]]
print(f)
GPMC().satcount( f )

[[4, 1], [-4, -1], [2, 4, -7], [3, 7, -10], [-2, -7], [-3, -10], [-4, 7], [-7, 10], [8, 4, 5], [11, 7, 8], [9, 5, 6], [12, 8, 9], [8, 2, 5], [11, 3, 8], [9, 2, 6], [12, 3, 9], [-8, -2, -4], [-11, -3, -7], [-9, -2, -5], [-12, -3, -8], [-8, -5], [-11, -8], [-9, -6], [-12, -9], [8, -2, -4], [12, -3, -8], [-8, 4], [-12, 8], [-8, 2], [-12, 3], [-12], [-1], [-2], [3]]


3